In [1]:
import pandas as pd
from functools import wraps
import json

# Dictionary to store transformation history 
transform_log = {}

def log_transform(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        data = args[0]
        transform_name = func.__name__
        
        # Initialize a list to store transformation history for this function
        if transform_name not in transform_log:
            transform_log[transform_name] = []
        
        # Store copy of DataFrame prior to transformation
        transform_info = {
            "df_before": data.copy(),
            "args": args,
            "kwargs": kwargs
        }
        
        result = func(*args, **kwargs)
        
        # Store copy of DataFrame after transformation
        transform_info["df_after"] = result.copy()
        
        # Append the transformation info to the history list
        transform_log[transform_name].append(transform_info)
        
        # Save the log to the file after each transformation
        save_log_to_file()
        
        return result
    return wrapper

# Function to save the log to the file
def save_log_to_file():
    log_file_path = "test.txt"
    with open(log_file_path, "w") as log_file:
        for transform_name, history_list in transform_log.items():
            log_file.write(f"Transformation: {transform_name}\n")
            
            for index, transform_info in enumerate(history_list, start=1):
                log_file.write(f"  Transformation #{index}:\n")
                log_file.write("  Arguments:\n")
                for arg in transform_info["args"]:
                    log_file.write(f"    {arg}\n")
                log_file.write("  Keyword Arguments:\n")
                for key, value in transform_info["kwargs"].items():
                    log_file.write(f"    {key}: {value}\n")
                log_file.write("  DataFrame Before:\n")
                log_file.write(f"{transform_info['df_before']}\n")
                log_file.write("  DataFrame After:\n")
                log_file.write(f"{transform_info['df_after']}\n")
                log_file.write("\n")

# Define transformation functions and apply them
@log_transform
def drop_columns(df, cols_to_drop):
    return df.drop(columns=cols_to_drop)

@log_transform  
def rename_columns(df, rename_dict):
    return df.rename(columns=rename_dict)

# Initial DataFrame
df = pd.DataFrame({"A": [1, 2], "B": [3, 4]}) 

# Infinite loop to listen for user input
while True:
    print("Available options:")
    print("1. drop_columns")
    print("2. rename_columns")
    print("3. Exit")
    choice = input("Enter your choice (1/2/3): ")

    if choice == "1":
        cols_to_drop = input("Enter columns to drop (comma-separated): ").split(",")
        df = drop_columns(df, cols_to_drop)
    elif choice == "2":
        rename_dict = json.loads(input("Enter dictionary for renaming columns: "))
        df = rename_columns(df, rename_dict)
    elif choice == "3":
        break
    else:
        print("Invalid choice. Please try again.")

print("Exiting the loop. The log has been saved to 'transform_log.txt'.")


Available options:
1. drop_columns
2. rename_columns
3. Exit
Enter your choice (1/2/3): 1
Enter columns to drop (comma-separated): A
Available options:
1. drop_columns
2. rename_columns
3. Exit


KeyboardInterrupt: Interrupted by user

In [3]:
df

,A,B
0,1,3
1,2,4


In [2]:
df

,B
0,3
1,4
